# Preparing basic tools

In [1]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

If the image is 8-bit unsigned, it is displayed as is.
<br>If the image is 16-bit unsigned or 32-bit integer, the pixels are divided by 256. That is, the value range [0,255*256] is mapped to [0,255].
<br>If the image is 32-bit floating-point, the pixel values are multiplied by 255. **That is, the value range [0,1] is mapped to [0,255].**

In [2]:
def read_image(path : str):
    image = cv.imread(path)
    image //= 255 # see above
    grey_img = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    return grey_img, np.shape(grey_img)

In [3]:
def identify_classes(X):
    return np.unique(X)

In [4]:
def display_image(title:str, image):
    cv.imshow(title, image)
    cv.waitKey()
    cv.destroyAllWindows()

In [5]:
def bruit_gauss(X, m, n, cl1, cl2, m1, sig1, m2, sig2):
    gaussian_class_1 = np.random.normal(m1, sig1, (m,n))
    gaussian_class_1[X == cl2] = 0
    gaussian_class_2 = np.random.normal(m2, sig2, (m,n)) # will probably have to use np.where class == {1,2}
    gaussian_class_2[X == cl1] = 0
    noise = gaussian_class_1 + gaussian_class_2

    return noise

In [6]:
def error_rate(tolerance, A, B, m, n):
    return 1 - ((abs(A - B) <= tolerance).sum())/(m*n)

In [7]:
def evaluate_kmeans(k, A, B):
    shape = A.shape
    
    model = KMeans(n_clusters=k, random_state=0).fit(B.reshape(-1,1))
    values = model.cluster_centers_
    labels = model.labels_
    img_seg = values[labels].reshape(shape)
    
    return img_seg, error_rate(0.05, img_seg, A, shape[0], shape[1])

# Puisqu'il s'agit de segmenter ... segmentons !

In [8]:
path = './images_BW/cible2.bmp'
image, shape = read_image(path)
cl1, cl2 = identify_classes(image)
print(f'classe 1 : {cl1}, classe 2 : {cl2}')

classe 1 : 0, classe 2 : 1


In [9]:
display_image('image', 255*image)

---

In [10]:
noisy = (image + bruit_gauss(image, shape[0], shape[1],cl1, cl2, 0, 1, 0, 0)).clip(0,1)
noisy_1 = (noisy + bruit_gauss(image, shape[0], shape[1],cl1, cl2, 0, 0, 0, 1)).clip(0,1)
noice = [noisy, noisy_1]
combine_noisy = np.concatenate(tuple(noice), axis=1)

display_image('noisy', combine_noisy)

In [11]:
img_seg, error = evaluate_kmeans(2, image, noisy_1)

In [12]:
error

1.0

In [13]:
display_image('toz', np.concatenate((img_seg, noisy_1)))

---